In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.ml.classification import DecisionTreeClassifier,NaiveBayes,LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

import nltk
from nltk.corpus import stopwords
from pyspark.ml import PipelineModel
from pyspark.ml import Pipeline
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import NGram,OneHotEncoder, StringIndexer, VectorAssembler

In [ ]:
nltk.download('stopwords')

In [ ]:
spark = SparkSession.builder \
    .appName("text_classification_trainer") \
    .master("local") \
    .getOrCreate()

In [ ]:
df = spark.read.json("gs://bdl2021_final_project/yelp_train.json")

In [ ]:
data = df.select('text', 'stars').dropna()

In [ ]:
data.show(5)

In [ ]:
def build_twograms(inputCol=["text","stars"], n=2):
    tokenizer = [Tokenizer(inputCol="text", outputCol="words")]
    
    ngrams = [
    
        NGram(n=i, inputCol="words", outputCol="{0}_grams".format(i))
        for i in range(1, n + 1)
    ]
 
    cv = [
        CountVectorizer(vocabSize=1024,inputCol="{0}_grams".format(i),
            outputCol="{0}_tf".format(i))
        for i in range(1, n + 1)
    ]
    idf = [IDF(inputCol="{0}_tf".format(i), outputCol="{0}_tfidf".format(i), minDocFreq=5) for i in range(1, n + 1)]
 
    assembler = [VectorAssembler(
        inputCols=["{0}_tfidf".format(i) for i in range(1, n + 1)],
        outputCol="features"
    )]
     
    lr = [LogisticRegression(labelCol="stars", featuresCol="features",maxIter=20, regParam=0.1)]  
    return Pipeline(stages=tokenizer + ngrams + cv + idf+ assembler +lr)

In [ ]:
# print("doing..")
# twogram_pipelineFit = build_twograms().fit(data)
# print("doing..")
# twogram_pipelineFit.save('gs://murari/modelpath/')
# print("done!!!")

In [ ]:
model = PipelineModel.load('gs://murari/modelpath/') #Load model


In [ ]:
evaluatorMulti = MulticlassClassificationEvaluator(labelCol="stars", predictionCol="prediction")
predictionAndTarget = model.transform(data).select("stars", "prediction")

In [ ]:
print("doing...")
acc = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "accuracy"})
f1 = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "f1"})
print("Accuracy of logistic is = %g"% (acc))
print("F1 of logistic is = %g"% (f1))